# Постановка задачи

**Тема**: Анализ производительности алгоритма решения задачи достижимости.

**Цель**: Исследовать эффективность алгоритма для различных вариантов представления разреженных матриц и векторов, а также определить оптимальный размер стартового множества для достижения наилучшей производительности.

**Задачи**:
1. Исследование следующих задач достижимости:
    1. Достижимость между всеми парами вершин.
    2. Достижимость для каждой из заданного множества стартовых вершин.
2. Выбор оптимального для решения задачи представления разреженных матриц и векторов (csr, coo, dia, dok, lil).
3. Определение оптимального размера множества стартовых вершин.

# Описание решений
В рамках задачи будут исследованы два алгоритма решения задач достижимости: 
## Tensor based RPQ
Получает на вход граф, набор стартовых и конечных вершин, а также регулярное выражение. Строит конечные автоматы по графу и регулярному выражению. Затем с помощью транзитивного замыкания находит матрицу смежности, через которую и проверяется достижимость в пересечённом автомате.
## Multiple source BFS based RPQ
Получает на вход граф, набор стартовых и конечных вершин, а также регулярное выражение. Определяет достижимость через обход в ширину, основанный на линейной алгебре. Автомат графа и регулярного выражения одновременно обходятся, фронт записывает в виде матрицы, куда последовательно добавляются значения. Один раз перемножая эту матрицу на матрицу смежности, совершается один обход. Когда процесс обходов завершается (фронт становится пустым), нужные пары стартовых и конечных вершин достаются из матрицы с посещёнными состояниями.

# Набор данных
## Графы
- Граф [wc](https://formallanguageconstrainedpathquerying.github.io/CFPQ_Data/graphs/data/wc.html#wc) содержит небольшое число вершин (332) и рёбер (269), относительно других графов из набора.
- Граф [ls](https://formallanguageconstrainedpathquerying.github.io/CFPQ_Data/graphs/data/ls.html#ls) содержит среднее число вершин (1687) и рёбер (1453), относительно других графов из набора. 
## Запросы
Были выбраны так, чтобы охватить все общепринятые конструкции регулярных выражений
- 'a a a d d'
- 'a* d*'
- '(a | d)*'
- (a* | b*)

In [1]:
from grapher import GraphData
import cfpq_data

graphs = {'wc': GraphData('wc'), 'ls': GraphData('ls')}
regexes = ['a a a d d', 'a* d*', '(a | d)*', '(a* | b*)'] # same for both graphs

[2024-10-12 14:34:10]>INFO>Found graph with name='wc'
[2024-10-12 14:34:10]>INFO>Load archive graph_archive=PosixPath('/home/dmitriy/dev/formal-languages/.venv/lib/python3.12/site-packages/cfpq_data/data/graphs/wc.tar.gz')
[2024-10-12 14:34:10]>INFO>Unzip graph name='wc' to file graph=PosixPath('/home/dmitriy/dev/formal-languages/.venv/lib/python3.12/site-packages/cfpq_data/data/graphs/wc/wc.csv')
[2024-10-12 14:34:10]>INFO>Remove archive graph_archive=PosixPath('/home/dmitriy/dev/formal-languages/.venv/lib/python3.12/site-packages/cfpq_data/data/graphs/wc.tar.gz')
[2024-10-12 14:34:10]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f4d65df4ec0> from path=PosixPath('/home/dmitriy/dev/formal-languages/.venv/lib/python3.12/site-packages/cfpq_data/data/graphs/wc/wc.csv')
[2024-10-12 14:34:10]>INFO>Construct labels_frequency=defaultdict(<class 'int'>, {'a': 113, 'd': 156}) for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f4d65df4ec0>
[2024-10-12 

# Эксперимент
## Оборудование
- **OS**: Ubuntu 24.04.1 LTS on Windows 10 x86_64
- **CPU**: 11th Gen Intel i7-1165G7 (8) @ 2.803GHz
- **Memory**: 16 GB, 3200 MT/s, 2 slots
## Измерения
В ходе эксперимента отслеживались следующие параметры:
- Тип алгоритма
- Тип графа (wc, ls)
- Регулярное выражение
- Размер множества начальных вершин (5%, 10% 20%, 30% от общего числа вершин)
- Тип матриц и векторов (coo, csr, dia, dok, lil)
- Время выполнения на этих данных (в долях секунды)



In [12]:
from networkx import MultiDiGraph
from task5 import ms_bfs_based_rpq, tensor_based_rpq
from scipy.sparse import (
    coo_array, csr_array, dia_array, dok_array, lil_array,
    coo_matrix, csr_matrix, dia_matrix, dok_matrix, lil_matrix)
import time
import pandas as pd 
import random
from scipy import stats

sparse_types = [
  ('coo', coo_array, coo_matrix),
  ('csr', csr_array, csr_matrix),
  ('dia', dia_array, dia_matrix),
  ('dok', dok_array, dok_matrix),
  ('lil', lil_array, lil_matrix)]

results = {
    'Algo': [], 
    'Graph Name': [], 
    'Regex': [], 
    'Start Nodes': [],
    'Time': [], 
    'Data Type': []
}


for graph_name in graphs.keys():
    graph = graphs[graph_name].graph

    for regex in regexes:
        procs = [0.05, 0.1, 0.2, 0.3] # percentage of the number of nodes

        # list with start nodes counts
        start_nodes_count = [int(len(graph.nodes) * proc) for proc in procs]

        for size in start_nodes_count:
            start_nodes = cfpq_data.generate_multiple_source(graph, size)
            end_nodes = random.sample(list(start_nodes), random.randint(1, size))

            for sparse_type in sparse_types:
                # repeat several times
                tensor_time = []
                bfs_time = []

                for i in range(3):
                    t_start = time.perf_counter()
                    tensor_based_rpq(regex, graph, start_nodes, end_nodes, sparse_type[1], sparse_type[0])
                    t_end = time.perf_counter()

                    tensor_time.append(t_end - t_start)

                    t_start = time.perf_counter()
                    ms_bfs_based_rpq(regex, graph, start_nodes, end_nodes, sparse_type[2], sparse_type[1])
                    t_end = time.perf_counter()

                    bfs_time.append(t_end - t_start)

                # TODO
                results['Algo'].append('Tensor Based RPQ')
                results['Graph Name'].append(graph_name)
                results['Regex'].append(regex)
                results['Start Nodes'].append(str(size))
                results['Time'].append(str(t_end - t_start))
                results['Data Type'].append(sparse_type[0])

                results['Algo'].append('MS-BFS Based RPQ')
                results['Graph Name'].append(graph_name)
                results['Regex'].append(regex)
                results['Start Nodes'].append(str(size))
                results['Time'].append(str(t_end - t_start))
                results['Data Type'].append(sparse_type[0])

df = pd.DataFrame(data=results)
print(df)


[2024-10-12 14:45:44]>INFO>Generate set of source vertices of 16 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f4d65df4ec0> for multiple-source evaluation


KeyboardInterrupt: 